In [4]:
import numpy as np
import math as m


In [107]:
m_list = [16,32,64,128,256]
t_list = [.25,.5,.75,1]


In [134]:
def BinomialTree(E, S0, M, T, r, vol, option = 'call', risk_neutrality = 0):
    """
    E is our strike price
    S0 is initial price
    M is the number of steps
    T is our time to expiry
    r is our risk free rate
    vol is the volatility of the asset
    Choose option to be 'call' or 'put' to decide on the type of option wanted
    IF risk_neutrality = 0 we dont preserve it, if == 1 we do & if ==2 we use the Cox Ross Rubenstein Formula
    
    """
    dt = T/M
    #don't preserve risk neutrality
    if risk_neutrality == 0:
        u = np.exp(r*(dt)) * ( 1 + np.sqrt(np.exp(vol**2 *(dt))-1 ) )
        d = np.exp(r*(dt)) * ( 1 - np.sqrt(np.exp(vol**2 *(dt))-1 ) )
        p =1/2
        
    #Preserve risk neutrality
    elif risk_neutrality == 1:
        A = 1/2 * (np.exp((r+vol**2)*dt) +  np.exp(-r*dt))
        u = A + np.sqrt(A**2 -1)
        d = A - np.sqrt(A**2 -1)
        p = (np.exp(r*dt)-d)/(u-d)
    
    #Cox Ross Rubenstein formula
    elif risk_neutrality == 2:
        u = np.exp(vol* np.sqrt(dt))
        d = np.exp(-vol* np.sqrt(dt))
        p = (np.exp(r*dt)-d )/(u-d)
        

            
    
    assetAtExpiry = (S0*d**np.arange(M,-1,-1)) * (u**np.arange(0,M+1,1))
    
    if option == 'call':
        optVal = np.maximum(assetAtExpiry-E,0)
    else:
        optVal = np.maximum(E- assetAtExpiry, 0)
        
    #Now compute the value of the call option at each stage
    for i in range(int(M)-1,-1,-1):   #we go to 0 here hence the 1st -1 and the 2nd -1 is because we are working backwards
        optVal = np.exp(-r*(dt)) * (p*optVal[range(1,i+2)] + (1-p)*optVal[range(0,i+1)])
        
    return optVal[0]
    

In [137]:
for t in t_list:
    my_list = []
    for m in m_list:
        my_list.append(round(BinomialTree(10,5,M = m,T = t,r = .12 ,vol = .5, option = 'put',risk_neutrality = 0),4))
    print(my_list)

[4.706, 4.7062, 4.7064, 4.7065, 4.7065]
[4.4479, 4.4523, 4.4517, 4.4521, 4.4525]
[4.2422, 4.2475, 4.2469, 4.2456, 4.2462]
[4.0749, 4.0687, 4.0748, 4.0725, 4.0731]


## 2nd type


In [138]:
for t in t_list:
    my_list = []
    for m in m_list:
        my_list.append(round(BinomialTree(10,5,M = m,T = t,r = .06 ,vol = .3, option = 'put',risk_neutrality = 1),4))
    print(my_list)

[4.8511, 4.8511, 4.8511, 4.8511, 4.8511]
[4.7046, 4.7047, 4.7047, 4.7048, 4.7048]
[4.5626, 4.5632, 4.5634, 4.5634, 4.5636]
[4.4292, 4.4299, 4.4299, 4.43, 4.4304]


## Cox Ross Rubenstein

In [136]:
for t in t_list:
    my_list = []
    for m in m_list:
        my_list.append(round(BinomialTree(10,5,M = m,T = t,r = .12 ,vol = .5, option = 'put',risk_neutrality = 2),4))
    print(my_list)

[4.706, 4.7062, 4.7063, 4.7065, 4.7065]
[4.4472, 4.4519, 4.4516, 4.4519, 4.4525]
[4.2394, 4.2461, 4.2461, 4.2451, 4.2462]
[4.0691, 4.0665, 4.0734, 4.0722, 4.0724]


In [5]:
def dFac(r,dt):
    return(np.exp(-r*dt))

In [6]:
S0=40; E=40; B=35; T=1; r=0.04; sig=0.4
M=400; dt=T/M
u=np.exp(sig*np.sqrt(dt));
d=np.exp(-sig*np.sqrt(dt));
p=(np.exp(r*dt)-d)/(u-d);
# Possible values of the underlying asset at expiration:
upwr=u**np.arange(0,M+1)
dpwr=d**np.arange(M,-1,-1)
assetAtExpiry=(S0*dpwr)*(upwr)


putVals = np.maximum(E-assetAtExpiry, 0)



# Stage 2 variation for down-and-out call option
callVals=np.maximum(assetAtExpiry-E,0)
for j in np.where(assetAtExpiry>=B):
    callVals[j]=0

# Stage 3 variation for down-and-out call option
for i in range(int(M)-1,-1,-1):
    valsUp=callVals[range(1,i+2)]
    valsDown=callVals[range(0,i+1)]
    callVals=dFac(r,dt)*(p*valsUp+(1-p)*valsDown)
    siValues=S0*upwr[range(0,i+1)]*dpwr[range(M-i,M+1)]
for j in np.where(siValues<B):
    callVals[j]=0
print(callVals[0])

0.0


---------------------------------------------------------------------